# Dataset Exploration - Digital Twin Power System Data

This notebook explores the digital twin dataset structure and visualizes sample data.

## Objectives:
1. Inspect repository structure and available files
2. Load sample data from different modalities
3. Visualize voltage magnitudes, phasor angles, and topology data
4. Assess data quality and characteristics

In [ ]:
# Import libraries
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

from data_loader import DigitalTwinDataLoader, load_sample_data

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("Libraries imported successfully")

## 1. Repository Structure Exploration

In [ ]:
# Explore dataset directory structure
dataset_path = "../digital-twin-dataset/digital-twin-dataset"

def print_directory_tree(path, prefix="", max_depth=3, current_depth=0):
    """Print directory tree structure."""
    if current_depth > max_depth:
        return
    
    path = Path(path)
    if not path.exists():
        print(f"Path does not exist: {path}")
        return
    
    items = sorted(path.iterdir())
    for i, item in enumerate(items):
        is_last = i == len(items) - 1
        current_prefix = "└── " if is_last else "├── "
        
        if item.is_dir():
            print(f"{prefix}{current_prefix}{item.name}/")
            next_prefix = prefix + ("    " if is_last else "│   ")
            print_directory_tree(item, next_prefix, max_depth, current_depth + 1)
        else:
            size_mb = item.stat().st_size / (1024 * 1024)
            print(f"{prefix}{current_prefix}{item.name} ({size_mb:.2f} MB)")

print("Digital Twin Dataset Structure:")
print_directory_tree(dataset_path)

## 2. Load and Inspect Sample Data

In [ ]:
# Initialize data loader
loader = DigitalTwinDataLoader(dataset_path)

# List available files
available_files = loader.list_available_files()

print("Available data files:")
for modality, files in available_files.items():
    print(f"\n{modality.upper()}:")
    if files:
        for file in files[:5]:  # Show first 5 files
            print(f"  - {file}")
        if len(files) > 5:
            print(f"  ... and {len(files) - 5} more files")
    else:
        print("  No files found (will use synthetic data)")

In [ ]:
# Load sample data
print("Loading sample data...")
data = load_sample_data(dataset_path, small_data_mode=True, synthetic_fallback=True)

print("\nLoaded data summary:")
for modality, df in data.items():
    print(f"\n{modality.upper()}:")
    print(f"  Shape: {df.shape}")
    print(f"  Time range: {df.index[0]} to {df.index[-1]}")
    print(f"  Duration: {df.index[-1] - df.index[0]}")
    print(f"  Columns: {list(df.columns[:5])}{'...' if len(df.columns) > 5 else ''}")
    print(f"  Data types: {df.dtypes.value_counts().to_dict()}")

## 3. Data Quality Assessment

In [ ]:
# Assess data quality
print("Data Quality Assessment:")
print("=" * 50)

for modality, df in data.items():
    print(f"\n{modality.upper()} QUALITY:")
    
    # Missing values
    missing_count = df.isnull().sum().sum()
    missing_pct = (missing_count / df.size) * 100
    print(f"  Missing values: {missing_count} ({missing_pct:.2f}%)")
    
    # Numeric columns statistics
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"  Numeric columns: {len(numeric_cols)}")
        print(f"  Mean range: [{df[numeric_cols].mean().min():.3f}, {df[numeric_cols].mean().max():.3f}]")
        print(f"  Std range: [{df[numeric_cols].std().min():.3f}, {df[numeric_cols].std().max():.3f}]")
        
        # Check for constant columns
        constant_cols = df[numeric_cols].columns[df[numeric_cols].std() == 0]
        if len(constant_cols) > 0:
            print(f"  Constant columns: {len(constant_cols)}")
    
    # Time series regularity
    time_diffs = df.index.to_series().diff().dropna()
    if len(time_diffs) > 0:
        mode_diff = time_diffs.mode()[0] if len(time_diffs.mode()) > 0 else time_diffs.median()
        irregular_count = (time_diffs != mode_diff).sum()
        print(f"  Time regularity: {len(time_diffs) - irregular_count}/{len(time_diffs)} regular intervals")
        print(f"  Sampling rate: ~{mode_diff.total_seconds():.1f} seconds")

## 4. Data Visualization

In [ ]:
# Create comprehensive visualization
n_modalities = len(data)
fig, axes = plt.subplots(n_modalities, 2, figsize=(15, 5*n_modalities))
if n_modalities == 1:
    axes = axes.reshape(1, -1)

for i, (modality, df) in enumerate(data.items()):
    # Time series plot
    ax1 = axes[i, 0]
    cols_to_plot = df.columns[:min(5, len(df.columns))]
    for col in cols_to_plot:
        ax1.plot(df.index, df[col], label=col, alpha=0.7, linewidth=1)
    
    ax1.set_title(f'{modality.capitalize()} - Time Series')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Value')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # Distribution plot
    ax2 = axes[i, 1]
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        # Plot histograms for first few numeric columns
        cols_to_hist = numeric_cols[:min(3, len(numeric_cols))]
        for col in cols_to_hist:
            ax2.hist(df[col].dropna(), bins=30, alpha=0.6, label=col, density=True)
        
        ax2.set_title(f'{modality.capitalize()} - Distributions')
        ax2.set_xlabel('Value')
        ax2.set_ylabel('Density')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    else:
        ax2.text(0.5, 0.5, 'No numeric data\nfor distribution plot', 
                ha='center', va='center', transform=ax2.transAxes)
        ax2.set_title(f'{modality.capitalize()} - No Numeric Data')

plt.tight_layout()
plt.show()

## 5. Power System Specific Analysis

In [ ]:
# Analyze voltage-like measurements (assuming magnitude data represents voltages)
if 'magnitude' in data:
    magnitude_data = data['magnitude']
    
    print("Voltage Magnitude Analysis:")
    print("=" * 30)
    
    # Voltage statistics
    voltage_cols = [col for col in magnitude_data.columns if 'voltage' in col.lower()]
    if not voltage_cols:
        voltage_cols = magnitude_data.select_dtypes(include=[np.number]).columns[:5]
    
    if len(voltage_cols) > 0:
        voltage_stats = magnitude_data[voltage_cols].describe()
        print(f"Voltage-like measurements statistics:")
        print(voltage_stats)
        
        # Check for voltage violations (assuming per-unit system)
        low_voltage_threshold = 0.95
        high_voltage_threshold = 1.05
        
        violations = {
            'low': (magnitude_data[voltage_cols] < low_voltage_threshold).sum().sum(),
            'high': (magnitude_data[voltage_cols] > high_voltage_threshold).sum().sum()
        }
        
        total_measurements = magnitude_data[voltage_cols].size
        print(f"\nVoltage Violations (assuming p.u. system):")
        print(f"  Low voltage (<{low_voltage_threshold}): {violations['low']} ({violations['low']/total_measurements*100:.2f}%)")
        print(f"  High voltage (>{high_voltage_threshold}): {violations['high']} ({violations['high']/total_measurements*100:.2f}%)")

# Analyze phasor data if available
if 'phasor' in data:
    phasor_data = data['phasor']
    
    print("\nPhasor Angle Analysis:")
    print("=" * 25)
    
    angle_cols = [col for col in phasor_data.columns if 'angle' in col.lower()]
    if not angle_cols:
        angle_cols = phasor_data.select_dtypes(include=[np.number]).columns[:3]
    
    if len(angle_cols) > 0:
        angle_stats = phasor_data[angle_cols].describe()
        print(f"Angle measurements statistics (radians):")
        print(angle_stats)
        
        # Angle differences (system stability indicator)
        if len(angle_cols) > 1:
            angle_diffs = phasor_data[angle_cols].diff(axis=1).iloc[:, 1:]
            max_angle_diff = angle_diffs.abs().max().max()
            print(f"\nMaximum angle difference: {max_angle_diff:.3f} radians ({np.degrees(max_angle_diff):.1f} degrees)")

# Analyze topology data
if 'topology' in data:
    topology_data = data['topology']
    
    print("\nTopology Analysis:")
    print("=" * 20)
    
    # Switching events
    binary_cols = [col for col in topology_data.columns if topology_data[col].nunique() <= 2]
    
    if binary_cols:
        print(f"Binary topology features: {len(binary_cols)}")
        
        # Count switching events
        switching_events = 0
        for col in binary_cols:
            changes = (topology_data[col].diff() != 0).sum()
            switching_events += changes
        
        print(f"Total switching events detected: {switching_events}")
        
        # Show current topology state
        current_state = topology_data[binary_cols].iloc[-1]
        print(f"\nCurrent topology state:")
        for col, state in current_state.items():
            status = "CLOSED" if state == 1 else "OPEN"
            print(f"  {col}: {status}")

## 6. Summary and Recommendations

In [ ]:
# Generate summary report
print("DATASET EXPLORATION SUMMARY")
print("=" * 50)

total_samples = sum(len(df) for df in data.values())
total_features = sum(len(df.columns) for df in data.values())
time_span = max(df.index[-1] for df in data.values()) - min(df.index[0] for df in data.values())

print(f"Dataset Overview:")
print(f"  Modalities: {len(data)}")
print(f"  Total samples: {total_samples:,}")
print(f"  Total features: {total_features}")
print(f"  Time span: {time_span}")

print(f"\nData Quality:")
total_missing = sum(df.isnull().sum().sum() for df in data.values())
total_values = sum(df.size for df in data.values())
missing_pct = (total_missing / total_values) * 100 if total_values > 0 else 0
print(f"  Missing data: {missing_pct:.2f}%")
print(f"  Data completeness: {100 - missing_pct:.2f}%")

print(f"\nRecommendations for AIDM:")
print(f"  ✓ Data is suitable for time series analysis")
print(f"  ✓ Multiple modalities available for fusion")
if missing_pct < 5:
    print(f"  ✓ Low missing data rate - good for ML training")
else:
    print(f"  ⚠ Consider imputation strategies for missing data")

print(f"  ✓ Proceed with preprocessing pipeline")
print(f"  ✓ Ready for feature engineering and model training")

print("\nNext Steps:")
print(f"  1. Run notebook_1_preprocess.ipynb for data preprocessing")
print(f"  2. Generate attack datasets with notebook_2_attacks.ipynb")
print(f"  3. Train and evaluate AIDM with notebook_3_train_evaluate.ipynb")

print("\n" + "=" * 50)
print("EXPLORATION COMPLETED SUCCESSFULLY")
print("=" * 50)